In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression, mutual_info_regression, SelectFromModel, VarianceThreshold
from methods import pre


In [12]:
import xgboost as xgb
import lightgbm as lgb
from sklearn import linear_model
print(xgb.XGBRegressor.__name__)
print(linear_model.Lasso.__name__)

XGBRegressor
Lasso


## Reducing memory usage by using indices

In [2]:
from classes import drug
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression, mutual_info_regression, SelectFromModel, VarianceThreshold

In [1]:
%%time
gdsc_ge = pd.read_csv('data/Processed/gdsc_cell_ge.csv').fillna(0).set_index('CCL')
ctrp_ge = pd.read_csv('data/Processed/ctrp_cell_ge.csv').fillna(0).set_index('CCL')
gdsc_dr = pd.read_csv('data/Processed/gdsc_poz_dr.csv').fillna(0)
ctrp_dr = pd.read_csv('data/Processed/ctrp_poz_dr.csv').fillna(0)

NameError: name 'pd' is not defined

In [3]:
%%time
aag = drug('17-AAG', {'ctrp': ctrp_ge, 'gdsc': gdsc_ge}, {'ctrp': ctrp_dr, 'gdsc': gdsc_dr})

CPU times: user 839 ms, sys: 88.7 ms, total: 928 ms
Wall time: 947 ms


In [4]:
aag.pre()

In [5]:
'ge' in aag.__dict__

False

In [6]:
aag.combine()

In [7]:
aag.split()

In [8]:
aag.get('X', 'train').shape

(827, 20600)

In [9]:

aag.fs(f_regression, n=0.1)

In [10]:
aag.get('X', 'train').shape

(827, 2060)

In [ ]:
aag.X['fs_test']

## Speeding up combine

In [12]:
gdsc_dr = pd.read_csv('data/Processed/gdsc_poz_dr.csv').fillna(0)
gdsc_ge = pd.read_csv('data/Processed/gdsc_cell_ge.csv').fillna(0).set_index('CCL')

In [13]:
gdsc_ge.shape[1]

19562

In [15]:
dr = gdsc_dr
ge = gdsc_ge

In [35]:
ge.join(dr[dr['Drug_name'] == '17-AAG'][['CCL', 'AUC_IC50']].set_index('CCL'), how='right')

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A3GALT2P,A4GALT,A4GNT,AAAS,AACS,...,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,AUC_IC50
CCL,,,,,,,,,,,,,,,,,,,,,
22RV1,3.537942,6.364651,5.332441,2.923030,2.815383,2.815383,3.241125,3.262633,4.722157,4.942126,...,7.526319,9.274166,3.785757,5.201929,2.742575,4.635015,3.940190,4.658663,8.088672,0.169156
2313287,3.370950,6.284884,3.485675,2.831562,2.913369,2.913369,3.313028,3.096527,4.873621,4.213177,...,6.526713,8.021952,3.520584,4.957371,2.636933,4.350120,4.395806,4.598583,7.951146,0.577314
5637,2.927335,2.892365,3.181651,2.926549,2.677943,2.677943,4.295357,3.205598,5.249042,4.495021,...,7.057572,9.261486,3.492602,4.813163,2.558705,4.580977,4.810975,4.371501,7.974367,0.828724
639V,3.953010,2.858072,2.892599,2.828334,2.729762,2.729762,2.960059,3.121154,4.445200,4.017422,...,6.310088,8.021780,3.919468,5.923131,2.782005,4.969272,5.481422,5.205446,8.118109,0.726141
647V,2.804009,2.944390,3.178071,2.675572,2.949402,2.949402,3.029297,3.024326,4.636786,4.724453,...,8.657911,9.426794,3.657576,6.082990,3.148358,5.182059,5.203430,4.806941,8.530606,0.008665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YAPC,3.043994,3.055992,3.532297,2.946112,2.835091,2.835091,3.273226,3.182387,4.474313,4.238488,...,8.619045,10.392332,3.704526,5.128979,3.082707,4.061633,3.664478,4.276836,7.238275,0.502122
YH13,5.167861,3.072842,3.049070,3.098071,3.029716,3.029716,2.990092,3.318574,4.561866,3.581945,...,8.048126,8.235304,3.305898,4.745612,2.577626,5.215836,7.003537,4.203823,7.919283,0.881166
YKG1,4.711624,3.067945,3.306930,3.037960,2.713442,2.713442,3.154398,2.988924,4.586814,3.781220,...,7.978261,9.150385,3.972405,5.166082,2.813449,5.117194,5.085635,4.918170,8.208343,0.803193


In [28]:
dr[dr['Drug_name'] == '17-AAG'][['CCL', 'AUC_IC50']].set_index('CCL')

,AUC_IC50
CCL,
22RV1,0.169156
2313287,0.577314
5637,0.828724
639V,0.726141
647V,0.008665
...,...
YAPC,0.502122
YH13,0.881166
YKG1,0.803193


In [16]:
%%time
drug_dr = dr[dr['Drug_name'] == '17-AAG'][['CCL', 'AUC_IC50']]

CPU times: user 7.94 ms, sys: 1.2 ms, total: 9.14 ms
Wall time: 15.4 ms


In [17]:
%%time
X = np.array([list(ge.loc[i].values) for i in drug_dr[drug_dr['CCL'].isin(ge.index)]['CCL']])

CPU times: user 2.9 s, sys: 258 ms, total: 3.16 s
Wall time: 3.33 s


In [18]:
%%time
X = X.reshape(drug_dr.shape[0], ge.shape[1])

CPU times: user 31 µs, sys: 1 µs, total: 32 µs
Wall time: 35 µs


In [19]:
%%time
data = pd.DataFrame(X).assign(DR = drug_dr['AUC_IC50'])

CPU times: user 48.3 ms, sys: 9.44 ms, total: 57.7 ms
Wall time: 66.9 ms


In [66]:
data

,0,1,2,3,4,5,6,7,8,9,...,19553,19554,19555,19556,19557,19558,19559,19560,19561,DR
0,3.537942,6.364651,5.332441,2.923030,2.815383,2.815383,3.241125,3.262633,4.722157,4.942126,...,7.526319,9.274166,3.785757,5.201929,2.742575,4.635015,3.940190,4.658663,8.088672,0.169156
1,3.370950,6.284884,3.485675,2.831562,2.913369,2.913369,3.313028,3.096527,4.873621,4.213177,...,6.526713,8.021952,3.520584,4.957371,2.636933,4.350120,4.395806,4.598583,7.951146,0.577314
2,2.927335,2.892365,3.181651,2.926549,2.677943,2.677943,4.295357,3.205598,5.249042,4.495021,...,7.057572,9.261486,3.492602,4.813163,2.558705,4.580977,4.810975,4.371501,7.974367,0.828724
3,3.953010,2.858072,2.892599,2.828334,2.729762,2.729762,2.960059,3.121154,4.445200,4.017422,...,6.310088,8.021780,3.919468,5.923131,2.782005,4.969272,5.481422,5.205446,8.118109,0.726141
4,2.804009,2.944390,3.178071,2.675572,2.949402,2.949402,3.029297,3.024326,4.636786,4.724453,...,8.657911,9.426794,3.657576,6.082990,3.148358,5.182059,5.203430,4.806941,8.530606,0.008665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,3.043994,3.055992,3.532297,2.946112,2.835091,2.835091,3.273226,3.182387,4.474313,4.238488,...,8.619045,10.392332,3.704526,5.128979,3.082707,4.061633,3.664478,4.276836,7.238275,0.502122
656,5.167861,3.072842,3.049070,3.098071,3.029716,3.029716,2.990092,3.318574,4.561866,3.581945,...,8.048126,8.235304,3.305898,4.745612,2.577626,5.215836,7.003537,4.203823,7.919283,0.881166
657,4.711624,3.067945,3.306930,3.037960,2.713442,2.713442,3.154398,2.988924,4.586814,3.781220,...,7.978261,9.150385,3.972405,5.166082,2.813449,5.117194,5.085635,4.918170,8.208343,0.803193
658,3.143593,3.779792,5.095163,2.879333,2.749091,2.749091,3.186225,3.359589,4.511721,4.241912,...,8.196766,9.205680,3.515323,5.152341,2.472749,5.046783,6.016707,5.168183,7.063949,0.513477


## Speeding up pre-processing

In [2]:
gdsc_ge = pd.read_csv('data/Processed/gdsc_cell_ge.csv').fillna(0).set_index('CCL')

In [3]:
print(type(gdsc_ge.keys()))

<class 'pandas.core.indexes.base.Index'>


In [5]:
gdsc_ge.shape

(706, 19562)

In [6]:
gdsc_ge.shape[0]

706

In [8]:
%%time
under = (gdsc_ge.to_numpy()>5).T.astype(np.int8)

n = [np.count_nonzero(i) > 0.1*gdsc_ge.shape[0] for i in under]
names = {gdsc_ge.keys()[k]:v for k, v in enumerate(n)}
indices = [k for k,v in names.items() if v]
index = gdsc_ge[indices]

CPU times: user 117 ms, sys: 6.79 ms, total: 124 ms
Wall time: 125 ms


In [9]:
%%time
index2 = pre(gdsc_ge,t=5, p=0.1)

CPU times: user 6.17 s, sys: 127 ms, total: 6.3 s
Wall time: 6.36 s


In [10]:
index.shape

(706, 9919)

In [11]:
index2.shape

(706, 9919)

## Domain Adaptation experiments

In [113]:
X_a = np.random.rand(3, 2) * 3

In [114]:
X_b = np.random.rand(3, 2) * 4

In [115]:
X_c = np.random.rand(3, 2) * 5

In [116]:
def jump(domain, n, data):
    result = []
    for i, ele in enumerate(data):
        result.append(ele)
        for j in range(0, domain):
            result.append(0)
        result.append(ele)
        for j in range(0, n-domain-1):
            result.append(0)
    return result
    

In [117]:
def feda(domains):
    n = len(domains)
    
    samples = 0
    for i in domains:
        samples += i.shape[0]
    
    features = domains[0].shape[1]*(n+1)
    
    new = np.zeros(features)
    for i, data in enumerate(domains):
        for j in data:
            new = np.vstack([new, jump(i, n, j)])
            
    return new[1:]

In [118]:
feda([X_a, X_b, X_c])

array([[2.39552445, 2.39552445, 0.        , 0.        , 0.8923882 ,
        0.8923882 , 0.        , 0.        ],
       [2.09713092, 2.09713092, 0.        , 0.        , 1.99024727,
        1.99024727, 0.        , 0.        ],
       [2.17421497, 2.17421497, 0.        , 0.        , 2.66618415,
        2.66618415, 0.        , 0.        ],
       [3.77029806, 0.        , 3.77029806, 0.        , 0.52768015,
        0.        , 0.52768015, 0.        ],
       [3.03560501, 0.        , 3.03560501, 0.        , 1.3673342 ,
        0.        , 1.3673342 , 0.        ],
       [2.41889914, 0.        , 2.41889914, 0.        , 3.77142649,
        0.        , 3.77142649, 0.        ],
       [0.70066211, 0.        , 0.        , 0.70066211, 3.89969596,
        0.        , 0.        , 3.89969596],
       [1.67451535, 0.        , 0.        , 1.67451535, 0.97901028,
        0.        , 0.        , 0.97901028],
       [0.52645206, 0.        , 0.        , 0.52645206, 1.13973433,
        0.        , 0.      

## Mess

In [48]:
from methods import fs

In [49]:
fs.__doc__

' Returns a subset of {X_train} and {X_test} with features being selected by the method {model}\n    :param int n: it can be the variance thereshold or the number of chosen features \n    :\n    '

In [1]:
from classes import drug
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import config as c
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression, mutual_info_regression, SelectFromModel, VarianceThreshold

In [2]:
gdsc_ge = pd.read_csv(c.dir + 'gdsc_cell_ge.csv').fillna(0).set_index('CCL')
ctrp_ge = pd.read_csv(c.dir + 'ctrp_cell_ge.csv').fillna(0).set_index('CCL')
gdsc_dr = pd.read_csv(c.dir + 'gdsc_poz_dr.csv').fillna(0)
ctrp_dr = pd.read_csv(c.dir + 'ctrp_poz_dr.csv').fillna(0)

In [3]:
%%time
aag = drug('17-AAG', {'ctrp': ctrp_ge, 'gdsc': gdsc_ge}, {'ctrp': ctrp_dr, 'gdsc': gdsc_dr})

CPU times: user 886 ms, sys: 94.7 ms, total: 980 ms
Wall time: 983 ms


In [4]:
aag.to_json()

col
<class 'list'>
da
<class 'dict'>
data
<class 'pandas.core.frame.DataFrame'>
dr
<class 'pandas.core.frame.DataFrame'>
ge
<class 'pandas.core.frame.DataFrame'>
name
<class 'str'>
predicted
<class 'list'>


In [5]:
%%time
aag.pre()

CPU times: user 490 ms, sys: 294 ms, total: 784 ms
Wall time: 864 ms


In [6]:
%%time
aag.combine()

CPU times: user 477 ms, sys: 364 ms, total: 842 ms
Wall time: 1.01 s


In [7]:
%%time
aag.split()

CPU times: user 138 ms, sys: 101 ms, total: 240 ms
Wall time: 242 ms


In [8]:
%%time
aag.fs(f_regression, n=0.01)

CPU times: user 335 ms, sys: 166 ms, total: 502 ms
Wall time: 431 ms


In [66]:
%%time
aag.feda()

CPU times: user 905 ms, sys: 573 ms, total: 1.48 s
Wall time: 1.5 s


In [9]:
aag.get('X', 'train')

BZRAP1-AS1     IFITM3  LINC00528  LINC00959  LOC100505812  \
     CCL                                                                   
ctrp U2OS        4.360345  10.689561   3.691387   4.054192      5.356219   
gdsc CHP126      0.000000   3.340287   0.000000   0.000000      0.000000   
ctrp NCIN87      4.093948  11.816014   3.491945   5.378071      5.724491   
     PK1         3.830065  10.581781   3.416218   4.407161      4.923667   
gdsc NCIH2029    0.000000   7.795346   0.000000   0.000000      0.000000   
...                   ...        ...        ...        ...           ...   
ctrp KALS1       4.664948  11.255554   3.839601   4.325554      6.289857   
gdsc NCIH82      0.000000   3.079115   0.000000   0.000000      0.000000   
     SW948       0.000000   9.500209   0.000000   0.000000      0.000000   
ctrp KP4         4.358944  11.543933   3.874033   6.000472      4.276625   
     NCIH322     4.363853   9.139073   3.879351   4.053817      4.242413   

               LOC285957   MIR17HG       MYH9     MYO1C      RIN1  ...  \
     CCL                                                           ...   
ctrp U2OS       3.209209  3.133829  10.369628  9.099938  5.726059  ...   
gdsc CHP126     0.000000  6.322012   3.760066  4.122908  3.477277  ...   
ctrp NCIN87     3.334916  3.250277   9.794020  8.620573  5.785392  ...   
     PK1        3.231939  3.047338  10.381888  8.931581  6.357494  ...   
gdsc NCIH2029   0.000000  3.674309   5.355261  5.181834  3.363687  ...   
...                  ...       ...        ...       ...       ...  ...   
ctrp KALS1      3.344808  2.851211   9.974136  8.298545  5.088644  ...   
gdsc NCIH82     0.000000  8.078923   3.392254  4.937312  3.308779  ...   
     SW948      0.000000  5.916487   5.005639  6.591044  4.483314  ...   
ctrp KP4        3.239486  2.901072   9.184660  9.031135  5.742611  ...   
     NCIH322    3.696640  3.090428  10.440037  8.257365  5.979493  ...   

                ZNF271P    ZNF429    ZNF431  ZNF559-ZNF177  ZNF670-ZNF695  \
     CCL                                                                    
ctrp U2OS      0.000000  0.000000  0.000000       0.000000       0.000000   
gdsc CHP126    6.644506  6.422067  6.155632       5.531044       5.920411   
ctrp NCIN87    0.000000  0.000000  0.000000       0.000000       0.000000   
     PK1       0.000000  0.000000  0.000000       0.000000       0.000000   
gdsc NCIH2029  6.021848  4.798837  4.624152       4.032227       4.673281   
...                 ...       ...       ...            ...            ...   
ctrp KALS1     0.000000  0.000000  0.000000       0.000000       0.000000   
gdsc NCIH82    5.102663  4.317909  4.890870       4.251698       5.038659   
     SW948     4.807431  5.665950  5.753016       3.546495       4.612117   
ctrp KP4       0.000000  0.000000  0.000000       0.000000       0.000000   
     NCIH322   0.000000  0.000000  0.000000       0.000000       0.000000   

                 ZNF695    ZNF714    ZNF726    ZNF761   ZNF767P  
     CCL                                                         
ctrp U2OS      0.000000  0.000000  0.000000  0.000000  0.000000  
gdsc CHP126    5.656609  3.635006  3.739052  5.279030  4.104843  
ctrp NCIN87    0.000000  0.000000  0.000000  0.000000  0.000000  
     PK1       0.000000  0.000000  0.000000  0.000000  0.000000  
gdsc NCIH2029  3.120853  3.482195  7.263432  5.108294  4.197822  
...                 ...       ...       ...       ...       ...  
ctrp KALS1     0.000000  0.000000  0.000000  0.000000  0.000000  
gdsc NCIH82    4.651086  3.300885  7.956783  4.963780  3.845872  
     SW948     3.539816  3.585742  3.428558  4.744320  4.028150  
ctrp KP4       0.000000  0.000000  0.000000  0.000000  0.000000  
     NCIH322   0.000000  0.000000  0.000000  0.000000  0.000000  

[827 rows x 206 columns]

In [10]:
aag.data.drop(aag.metric, axis=1)

A1BG  A1BG-AS1      A1CF       A2M   A2M-AS1     A2ML1  \
     CCL                                                                  
ctrp 22RV1   4.753564  4.766881  7.445061  4.824879  5.714456  3.696364   
     42MGBA  7.290132  5.591604  4.030574  4.113296  3.277104  3.985902   
     5637    4.293473  4.510732  4.157908  4.110104  4.021707  4.169669   
     639V    6.627165  4.951915  4.094668  3.980078  4.311624  4.105991   
     697     6.991375  4.836096  4.027624  4.069556  4.350773  4.054177   
...               ...       ...       ...       ...       ...       ...   
gdsc YAPC    3.043994  0.000000  3.055992  3.532297  0.000000  2.946112   
     YH13    5.167861  0.000000  3.072842  3.049070  0.000000  3.098071   
     YKG1    4.711624  0.000000  3.067945  3.306930  0.000000  3.037960   
     YT      3.143593  0.000000  3.779792  5.095163  0.000000  2.879333   
     ZR7530  5.594177  0.000000  3.032126  3.562195  0.000000  3.108549   

                A2MP1    A4GALT     A4GNT      AA06  ...    ZNF773    ZNF812  \
     CCL                                             ...                       
ctrp 22RV1   4.500844  4.386674  3.917018  4.372562  ...  0.000000  0.000000   
     42MGBA  4.839848  4.578237  3.898615  4.481621  ...  0.000000  0.000000   
     5637    5.138686  6.044395  4.319823  4.457162  ...  0.000000  0.000000   
     639V    4.640191  4.547413  3.960633  4.682899  ...  0.000000  0.000000   
     697     4.691261  4.582222  3.989948  4.433872  ...  0.000000  0.000000   
...               ...       ...       ...       ...  ...       ...       ...   
gdsc YAPC    0.000000  3.273226  3.182387  0.000000  ...  5.275694  2.827617   
     YH13    0.000000  2.990092  3.318574  0.000000  ...  5.819260  3.036440   
     YKG1    0.000000  3.154398  2.988924  0.000000  ...  6.671234  2.813230   
     YT      0.000000  3.186225  3.359589  0.000000  ...  6.043061  2.985051   
     ZR7530  0.000000  3.611381  3.247737  0.000000  ...  3.619265  3.186172   

               ZNF816   ZNF833P    ZNF841    ZNF845    ZNF860  ZNRD1ASP  \
     CCL                                                                  
ctrp 22RV1   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
     42MGBA  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
     5637    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
     639V    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
     697     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
...               ...       ...       ...       ...       ...       ...   
gdsc YAPC    3.113137  3.413018  3.004792  2.991953  4.212500  4.518800   
     YH13    2.874351  3.359950  3.557884  3.183011  2.841925  4.917730   
     YKG1    3.140576  3.274792  3.544649  3.422049  2.635027  4.711590   
     YT      2.627298  3.170128  3.165902  3.156890  2.589527  3.915674   
     ZR7530  3.355644  3.386036  3.332804  3.944748  2.808594  4.772831   

                  ZP3      ZPR1  
     CCL                         
ctrp 22RV1   0.000000  0.000000  
     42MGBA  0.000000  0.000000  
     5637    0.000000  0.000000  
     639V    0.000000  0.000000  
     697     0.000000  0.000000  
...               ...       ...  
gdsc YAPC    5.097525  7.202968  
     YH13    4.791946  6.943269  
     YKG1    4.786302  6.520695  
     YT      4.119448  6.867354  
     ZR7530  6.543085  5.574740  

[1103 rows x 20600 columns]

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(aag.data.drop(aag.metric, axis=1), aag.data[aag.metric])

In [12]:
from methods import fs
X_train, X_test, co = fs(f_regression, X_train, X_test, y_train, n=0.01)

In [13]:
from methods import drp
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score

mod = drp(DecisionTreeRegressor(), X_train, y_train)
ypred= mod.predict(X_test)
r2_score(y_test, ypred)

-0.3029939981101124

In [14]:
%%time
aag.train(DecisionTreeRegressor())

CPU times: user 195 ms, sys: 85.5 ms, total: 280 ms
Wall time: 282 ms


In [15]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

aag.metrics([r2_score, mean_absolute_error, mean_squared_error, median_absolute_error])

{'r2_score': -0.5088792874287988,
 'mean_absolute_error': 0.28766421028763334,
 'mean_squared_error': 0.13503587014290738,
 'median_absolute_error': 0.2336659619395219}

In [16]:
aag.scores

AttributeError: 'drug' object has no attribute 'scores'

In [ ]:
aag.to_json()

## Runs

In [17]:
from runs import run
from sklearn.svm import SVR
from classes import tuning

In [18]:
from classes import drug
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression, mutual_info_regression, SelectFromModel, VarianceThreshold

In [19]:

#ge = {'ctrp': ctrp_ge, 'gdsc': gdsc_ge}
#dr = {'ctrp': ctrp_dr, 'gdsc': gdsc_dr}

t2 = {
    'degree': [2, 3, 4, 5],
    'epsilon' : [0.1, 0.2, 0.3, 0.9],
    'C':[0.01, 0.1, 1, 10, 100],
    'gamma':['scale']
}
tune = tuning(t2, iterations=50, cv=3, scoring='r2')

In [20]:
ge={'gdsc':True, 'ctrp':False, 'ccle':False}


In [21]:
%%time
r1, scores = run(ge, 'ElasticNet', False, 'DecisionTreeRegressor',drugs=3, n = 0.1, tuning = None)

CPU times: user 27.4 s, sys: 1.93 s, total: 29.4 s
Wall time: 54.9 s


In [34]:
for i in r1.values():
    print(len(i.col))


1607
1607
1607


In [26]:
model = {k:v.model.get_params() for k,v in r1.items()}

In [27]:
for i in r1.values():
    print(i.metrics([r2_score]))

{'r2_score': -0.4402398209397902}
{'r2_score': -1.1516677873876828}
{'r2_score': -0.9183615617612619}


In [34]:
import matplotlib.pyplot as plt
import seaborn as sns
scores = pd.DataFrame.from_dict(scores, orient='index')

In [35]:
scores = scores.join(pd.DataFrame.from_dict(model, orient='index'))

In [36]:
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression, mutual_info_regression, SelectFromModel, VarianceThreshold

In [37]:
VarianceThreshold.__name__

'VarianceThreshold'

In [38]:
scores

,r2_score,mean_absolute_error,mean_squared_error,median_absolute_error,criterion,max_depth,max_features,max_leaf_nodes,min_impurity_decrease,min_impurity_split,min_samples_leaf,min_samples_split,min_weight_fraction_leaf,presort,random_state,splitter
PD-173074,0.998486,0.001222,0.000030,0.000151,mse,None,None,None,0.0,None,1,2,0.0,False,None,best
S-Trityl-L-cysteine,0.999911,0.001753,0.000006,0.001241,mse,None,None,None,0.0,None,1,2,0.0,False,None,best
Vinorelbine,0.999976,0.000921,0.000002,0.000581,mse,None,None,None,0.0,None,1,2,0.0,False,None,best


In [7]:
a = [1,2,3]
b = [2,3,4]
x = [2,7,9]

In [8]:
c = set(a) & set(b)

In [9]:
list(c & set(x))

[2]

In [23]:
import pandas as pd
import config as c

In [24]:
ccle_ge = pd.read_csv(c.dir + 'ccle_cell_ge.csv').fillna(0).set_index('CCL')
ctrp_ge = pd.read_csv(c.dir + 'ctrp_cell_ge.csv').fillna(0).set_index('CCL')
ccle_dr = pd.read_csv(c.dir + 'ccle_poz_dr.csv').fillna(0)
ctrp_dr = pd.read_csv(c.dir + 'ctrp_poz_dr.csv').fillna(0)

In [26]:
drug_data = {'ccle':ccle_dr, 'ctrp':ctrp_dr}
[item for sublist in [list(j['Drug_name'].unique()) for j in drug_data.values()] for item in sublist]

['17-AAG',
 'AEW541',
 'Saracatinib',
 'Selumetinib',
 'Erlotinib',
 'Irinotecan',
 'L-685458',
 'Lapatinib',
 'LBW242',
 'Nilotinib',
 'Nutlin-3',
 'Paclitaxel',
 'Panobinostat',
 'PD-0325901',
 'PD-0332991',
 'Crizotinib',
 'PHA-665752',
 'PLX-4720',
 'RAF265',
 'Sorafenib',
 'TAE684',
 'Dovitinib',
 'Topotecan',
 'Vandetanib',
 '16-beta-bromoandrosterone',
 '1S,3R-RSL-3',
 '3-Cl-AHPC',
 '968',
 'A-804598',
 'AA-COCF3',
 'Abiraterone',
 'ABT-199',
 'ABT-737',
 'AC55649',
 'Afatinib',
 'AGK-2',
 'Alisertib',
 'Alisertib_navitoclax 2 to 1 ratio mol by mol',
 'Flavopiridol',
 'AM-580',
 'Apicidin',
 'AT-406',
 'AT13387',
 'AT7867',
 'Austocystin D',
 'Avicin D',
 'Avrainvillamide',
 'Axitinib',
 'AZ-3146',
 'Azacitidine',
 'AZD1480',
 'AZD4547',
 'AZD6482',
 'AZD7545',
 'AZD7762',
 'AZD8055',
 'B02',
 'Bafilomycin A1',
 'Barasertib',
 'Bardoxolone methyl',
 'Bax channel blocker',
 'BCL-LZH-4',
 'BEC',
 'Belinostat',
 'Bendamustine',
 'Betulinic acid',
 'Bexarotene',
 'BI-2536',
 'BIBR-1

## Fixing feda

In [35]:
from methods import feda
from classes import drug
import pandas as pd

In [34]:
ccle_ge = pd.read_csv(c.dir + 'ccle_cell_ge.csv').fillna(0).set_index('CCL')
ctrp_ge = pd.read_csv(c.dir + 'ctrp_cell_ge.csv').fillna(0).set_index('CCL')
ccle_dr = pd.read_csv(c.dir + 'ccle_poz_dr.csv').fillna(0)
ctrp_dr = pd.read_csv(c.dir + 'ctrp_poz_dr.csv').fillna(0)

In [48]:
a = ccle_ge[:3][['A1BG', 'A1CF']]
b = ctrp_ge[:3][['A1BG', 'A1CF']]
c = ctrp_ge[3:6][['A1BG', 'A1CF']]

In [49]:
a

,A1BG,A1CF
CCL,,
1321N1,5.542720,3.921564
22RV1,4.753564,7.445061
42MGBA,7.290132,4.030574


In [50]:
b

,A1BG,A1CF
CCL,,
22RV1,4.753564,7.445061
42MGBA,7.290132,4.030574
5637,4.293473,4.157908


In [51]:
c

,A1BG,A1CF
CCL,,
639V,6.627165,4.094668
697,6.991375,4.027624
769P,4.477092,5.060515


In [52]:
pd.DataFrame(feda([a.to_numpy(), b.to_numpy(), c.to_numpy()]))

,0,1,2,3,4,5,6,7
0,5.542720,5.542720,0.000000,0.000000,3.921564,3.921564,0.000000,0.000000
1,4.753564,4.753564,0.000000,0.000000,7.445061,7.445061,0.000000,0.000000
2,7.290132,7.290132,0.000000,0.000000,4.030574,4.030574,0.000000,0.000000
3,4.753564,0.000000,4.753564,0.000000,7.445061,0.000000,7.445061,0.000000
4,7.290132,0.000000,7.290132,0.000000,4.030574,0.000000,4.030574,0.000000
5,4.293473,0.000000,4.293473,0.000000,4.157908,0.000000,4.157908,0.000000
6,6.627165,0.000000,0.000000,6.627165,4.094668,0.000000,0.000000,4.094668
7,6.991375,0.000000,0.000000,6.991375,4.027624,0.000000,0.000000,4.027624
8,4.477092,0.000000,0.000000,4.477092,5.060515,0.000000,0.000000,5.060515
